In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import gzip
from collections import defaultdict
import math
import string
import pickle
from tqdm import tqdm
# import tensorflow as tf
from math import sin, cos, sqrt, atan2, radians
import numpy as np
from sklearn.neighbors import BallTree

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [3]:
places_file = "places.clean.json.gz"
users_file = "users.clean.json.gz"
reviews_file = "reviews.clean.json.gz"

In [4]:
places_data = []

In [5]:
for l in tqdm(parse(places_file)):
    places_data.append(l)
    
    

3114353it [03:35, 14477.74it/s]


In [6]:
places_map = {}
for p in tqdm(places_data):
    places_map[p['gPlusPlaceId']] = p

100%|█████████████████████████████| 3114353/3114353 [00:13<00:00, 228662.73it/s]


In [7]:
with open("us_reviews_restuarants_train.pkl",'rb') as f:
    us_restaurant_train = pickle.load(f)

In [8]:
with open("us_reviews_restuarants_test.pkl",'rb') as f:
    us_restaurant_test = pickle.load(f)

In [9]:
def user_item_interactions(rev_data):
    usersPerItem = defaultdict(set)
    itemsPerUser = defaultdict(set)
    reviewsPerUser = defaultdict(list)
    reviewsPerItem = defaultdict(list)
    ratingsDict = defaultdict(float)
    for d in tqdm(rev_data):
        u = d['gPlusUserId']
        i = d['gPlusPlaceId']
        ratingsDict[(u,i)] = d['rating']
        usersPerItem[i].add(u)
        itemsPerUser[u].add(i)
        reviewsPerUser[u].append(d)
        reviewsPerItem[i].append(d)
    return usersPerItem, itemsPerUser, reviewsPerUser, reviewsPerItem, ratingsDict


In [10]:
usersPerItem, itemsPerUser, reviewsPerUser, reviewsPerItem, ratingsDict = user_item_interactions(us_restaurant_train)

100%|███████████████████████████████| 454830/454830 [00:01<00:00, 433879.31it/s]


In [11]:
places_set = set(usersPerItem.keys())

In [12]:
mod_places_data = []
for d in tqdm(places_data):
    if d['gPlusPlaceId'] in places_set:
        mod_places_data.append(d)

100%|█████████████████████████████| 3114353/3114353 [00:18<00:00, 166713.66it/s]


In [13]:
len(mod_places_data)

147792

In [14]:
def compute_dist(gps_p1, gps_p2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = np.radians(gps_p1[0])
    lon1 = np.radians(gps_p1[1])
    lat2 = np.radians(gps_p2[0])
    lon2 = np.radians(gps_p2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
place_nbd_dict = defaultdict(list)
places_ids = np.array([d['gPlusPlaceId'] for d in mod_places_data])
places_gps = [d['gps'] for d in mod_places_data]
places_gps = np.array(places_gps)

In [16]:
R = 6371.0087714
a = np.array(places_gps)
X = np.radians(a.reshape(len(a),2))
tree = BallTree(X, leaf_size=2, metric='haversine')              

def compute_geodesic_distance(place_gps1, neighbours):
    b = np.array(place_gps1)
    dist, ind = tree.query(np.radians(b.reshape(1, -1)), k=neighbours)                
    x = (dist*R) 
    return x, ind

In [17]:
place_nbd_dict = defaultdict(list)
places_gps = [d['gps'] for d in mod_places_data]
places_gps = np.array(places_gps)
for p in tqdm(range(len(mod_places_data))):
    gps_p = mod_places_data[p]['gps']
    x, ind = compute_geodesic_distance(gps_p, 11)
    place_nbd_ids = places_ids[ind].tolist()[0]
    x = x.tolist()[0]
    place_nbd_dict[mod_places_data[p]['gPlusPlaceId']] = list(zip(place_nbd_ids[1:], x[1:]))

100%|█████████████████████████████████| 147792/147792 [00:23<00:00, 6299.97it/s]


In [18]:
a = [1,2,3]
b = [0.1,0.2,0.3]
list(zip(a,b))

[(1, 0.1), (2, 0.2), (3, 0.3)]

In [23]:
place_nbd_dict

defaultdict(list,
            {'100327153115986850675': [('103154435820053835305',
               0.021861547296399404),
              ('104635758967952741463', 1.1287298971006112),
              ('114337891141986309006', 1.2008434464934115),
              ('108745776275865721349', 1.346114527123214),
              ('100327524051176304229', 1.4106927105243152),
              ('113946334383741525230', 1.4669059700166796),
              ('105961177050105305675', 1.486827065939288),
              ('107133466275410087661', 1.5266343232922348),
              ('110498512440598360038', 1.67940333095524),
              ('102745521403443141668', 1.6972243795118844)],
             '106432060150136868000': [('112658258222599985090',
               0.18700396510667908),
              ('110957279608241057536', 0.5468789549044276),
              ('104992931640181349473', 0.5828679667811095),
              ('114725447033422635345', 0.6966011582541957),
              ('103213209045649348929', 0.751366

In [24]:
places_map['100327153115986850675']

{'name': "T C's Referee Sports Bar",
 'price': '$$',
 'address': ['5322 W 26th St', 'Sioux Falls, SD 57106'],
 'hours': [['Monday', [['11:00 am--10:00 pm']]],
  ['Tuesday', [['11:00 am--10:00 pm']], 1],
  ['Wednesday', [['11:00 am--10:00 pm']]],
  ['Thursday', [['11:00 am--10:00 pm']]],
  ['Friday', [['11:00 am--1:00 am']]],
  ['Saturday', [['11:00 am--1:00 am']]],
  ['Sunday', [['11:00 am--10:00 pm']]]],
 'phone': '(605) 361-2208',
 'closed': False,
 'gPlusPlaceId': '100327153115986850675',
 'gps': [43.529494, -96.792244]}

In [25]:
places_map['103154435820053835305']

{'name': 'Dynasty Chinese & Vietnamese Cuisine Restaurant',
 'price': '$$',
 'address': ['5326 W 26th St', 'Sioux Falls, SD 57106'],
 'hours': [['Monday', [['11:00 am--9:30 pm']]],
  ['Tuesday', [['11:00 am--9:30 pm']]],
  ['Wednesday', [['11:00 am--9:30 pm']], 1],
  ['Thursday', [['11:00 am--9:30 pm']]],
  ['Friday', [['11:00 am--10:30 pm']]],
  ['Saturday', [['11:00 am--10:30 pm']]],
  ['Sunday', [['11:00 am--8:30 pm']]]],
 'phone': '(605) 361-7788',
 'closed': False,
 'gPlusPlaceId': '103154435820053835305',
 'gps': [43.529688, -96.792288]}

In [26]:
a = [[3,4], [5,12], [6,8]]

In [27]:
b = np.asarray(a,dtype=float)

In [29]:
sum_distances = [3,4,5,6,7,8,9,10,11,12]
sum_distances = np.asarray(sum_distances,dtype=float)
# sum_distances = tf.Variable(sum_distances)

In [30]:
sum_distances

array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [31]:
sample_ind = [1,2,3]

In [32]:
# p = tf.convert_to_tensor(sample_ind, dtype=tf.int32)

In [33]:
# p

In [28]:
# selected_sums = tf.nn.embedding_lookup(sum_distances, p)

In [40]:
places_sum = []
for p in place_nbd_dict:
    sd = 0
    neighbours = place_nbd_dict[p]
    for n in neighbours:
        sd += n[1]
    places_sum.append(sd)
places_sum

[12.96523719825328,
 8.241445946183,
 101.51433284011027,
 2.81917692861589,
 8.889534719396899,
 11.603965589943249,
 20.416775514980984,
 6.363698537247075,
 10.02819974786495,
 2.026955481911381,
 39.21722584986401,
 3.6774800853877734,
 3.562274998546814,
 12.816420562757667,
 3.2771321239629,
 56.1339290442404,
 17.724391314865763,
 342.5426663965028,
 1.9596522495527642,
 3.0686631674512235,
 0.9803696220101572,
 0.5825484813903364,
 3.3861247492783244,
 26.48247366176802,
 23.978244234812145,
 12.120854429645814,
 32.59642509201169,
 4.641639818946614,
 5.202063715114296,
 2.3947271511781767,
 1.3651806819920527,
 20.229909766347063,
 8.10695151915004,
 29.905798956575428,
 7.272774227978672,
 0.8403474728125859,
 8.47742144574734,
 2.192128665657912,
 12.966426251100156,
 7.452081222810438,
 0.736988802254571,
 14.210330930232256,
 11.848463678974445,
 0.5991973491179468,
 6.129798762543509,
 24.635976238111375,
 13.277215449170694,
 2.7927878684015583,
 3.2620509632163315,
 10

In [38]:
places_idx = {}
for i,p in enumerate(place_nbd_dict):
    places_idx[p] = i
places_idx

{'100327153115986850675': 0,
 '106432060150136868000': 1,
 '100184392614713668281': 2,
 '110300304875024740707': 3,
 '115097126300227315960': 4,
 '112186783627869608416': 5,
 '113677754356449241397': 6,
 '107791259980241216475': 7,
 '113140673306093175878': 8,
 '105235564497581302433': 9,
 '108250088873819445455': 10,
 '103849800212863124922': 11,
 '108650735759424556977': 12,
 '115757957627721988675': 13,
 '104469148888717134733': 14,
 '107524535486760950602': 15,
 '112123355218244694227': 16,
 '107496667371371832451': 17,
 '100664388958324368598': 18,
 '100652111090931854161': 19,
 '115622769881026234674': 20,
 '109867835269515778588': 21,
 '100776794109208708781': 22,
 '113171419156445068202': 23,
 '108669557802494018449': 24,
 '107495784855044281180': 25,
 '109942353581026453111': 26,
 '112294156224902332705': 27,
 '101287866841003338666': 28,
 '108216095399410140928': 29,
 '108790374366638107750': 30,
 '107041613835869828931': 31,
 '107524296851262141691': 32,
 '103001509130786807

In [43]:
places_dist_dict = np.zeros((len(place_nbd_dict), len(place_nbd_dict)))
print(places_dist_dict.shape)

(147793, 147793)


In [47]:
for p in place_nbd_dict:
    row = places_idx[p]
    neighbours = place_nbd_dict[p]
    for n in neighbours:
        pl = n[0]
        d = n[1]
        idx = places_idx[pl]
        places_dist_dict[row,idx] = d

array([0., 0., 0., ..., 0., 0., 0.])